<a href="https://colab.research.google.com/github/ManuSinghYadav/Andrej_Karpathy_Zero_to_Hero/blob/main/AK_Lecture_Series_Nano_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-07 11:08:18--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-07-07 11:08:19 (16.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [ ]:
print(f"{len(text): ,}")

 1,115,394


In [ ]:
# Unique Charecters
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [ ]:
''.join(chars)

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [ ]:
# @title Mapping (for both encoding and decoding)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [ ]:
# @title Encode entire dataset
data = torch.tensor(encode(text))
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [ ]:
# @title Train-test split
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [ ]:
# @title Context window setup
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
print(f"x: {x} \ny: {y}")

x: tensor([18, 47, 56, 57, 58,  1, 15, 47]) 
y: tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [ ]:
# @title Batch setup
torch.manual_seed(1337)
batch_size = 4

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [ ]:
xb, yb = get_batch('train')
xb, yb

(tensor([[24, 43, 58,  5, 57,  1, 46, 43],
         [44, 53, 56,  1, 58, 46, 39, 58],
         [52, 58,  1, 58, 46, 39, 58,  1],
         [25, 17, 27, 10,  0, 21,  1, 54]]),
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
         [53, 56,  1, 58, 46, 39, 58,  1],
         [58,  1, 58, 46, 39, 58,  1, 46],
         [17, 27, 10,  0, 21,  1, 54, 39]]))

In [ ]:
# @title Bigram Setup
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    """idx (B, T), targets (B, T)."""
    logits = self.token_embedding_table(idx) # (B, T, C)
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    for i in range(max_new_tokens):
      logits, loss = self.forward(idx)
      logits = logits[:,-1,:] # Picking the last row of each batch.
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)
    return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m.forward(xb,yb)

output = m.generate(torch.tensor([[13]]), 100)
print(decode(output[0].tolist()))

AcGSdjZkr FIfV:jmhDbkiEV:R.h.jGbok'clQA:rE?LCrYN?ww3D$;s -VhE3Q'XVtEaMVo-TXTOvm3A3$VWPCFVgCvewDlIQ
UO


In [ ]:
# @title Training
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32

for i in range(10000):
  xb, yb = get_batch('train')
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  if i%500 == 0:
    print(f"{i:4d} / 1000 : {loss.item(): 3f}")

  if i == 8000:
    optimizer = torch.optim.AdamW(m.parameters(), lr=1e-5)  # Learning rate decay

output = m.generate(torch.tensor([[13]]), 500)
print(decode(output[0].tolist()))

   0 / 1000 :  4.658271
 500 / 1000 :  4.100937
1000 / 1000 :  3.786101
1500 / 1000 :  3.426409
2000 / 1000 :  3.134285
2500 / 1000 :  3.002464
3000 / 1000 :  2.658358
3500 / 1000 :  2.757277
4000 / 1000 :  2.605098
4500 / 1000 :  2.525246
5000 / 1000 :  2.579212
5500 / 1000 :  2.560179
6000 / 1000 :  2.538840
6500 / 1000 :  2.434014
7000 / 1000 :  2.433148
7500 / 1000 :  2.352535
8000 / 1000 :  2.408124
8500 / 1000 :  2.433011
9000 / 1000 :  2.507345
9500 / 1000 :  2.365025
Anng h hasbe pave pirance
RDe hicomyonthar's
PES:
AKEd itClel ourzincenonthioneir thondy, y heltieiengerofo'dsssit eFitcild pe wish3bevouprrurknthacnohathe; d!
MyWhind tt hinig t oPixxis es; st yo hind wotin grotonear 'so itJs jod weancotha:
h hay.JLile n prids, r loncave w hollular s O:
HIs; ht anjx?

DJe inqunt.

LaZEELAndathave l.
KEONH:
ARThanco be y,-JZNEEYowddy scat t tridesar, wyonthenous sels,&UMuresseys
PlorseelapinghienHen yof Gand m t l-h!E:
I hisgothers jveere!-e!
QUCotouciullle's fld


### Mathematical trick in Masking

We need to calculate the average of the previous tokens (we'll use softmax version).

In [ ]:
a = torch.ones(3, 3)
b = torch.randint(0,10,(3,2), dtype=torch.float)

print(a,'\n')
print(b,'\n')
print(a @ b)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]) 

tensor([[3., 0.],
        [9., 6.],
        [3., 0.]]) 

tensor([[15.,  6.],
        [15.,  6.],
        [15.,  6.]])


In [ ]:
# This is doing the sum of the previous tokens.
a = torch.tril(torch.ones(3, 3))
print(a,'\n')
print(b,'\n')
print(a @ b)

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]]) 

tensor([[3., 0.],
        [9., 6.],
        [3., 0.]]) 

tensor([[ 3.,  0.],
        [12.,  6.],
        [15.,  6.]])


In [ ]:
# Version 1: This is doing the avg of the previous tokens.
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, dim=1, keepdim=True)
print(a,'\n')
print(b,'\n')
print(a @ b)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]]) 

tensor([[3., 0.],
        [9., 6.],
        [3., 0.]]) 

tensor([[3., 0.],
        [6., 3.],
        [5., 2.]])


In [ ]:
# Version 2: Using softmax (In this we replaces the division)
a = torch.tril(torch.ones(3, 3))
a = a.masked_fill(a==0, float('-inf'))
F.softmax(a, -1)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

In [ ]:
# @title Self-Attention
head_size = 16
B, T, C = 4, 8, 32
x = torch.randn(B,T,C)

Query =  nn.Linear(C, head_size)
Key =  nn.Linear(C, head_size)
Value =  nn.Linear(C, head_size)

q = Query(x)
k = Key(x)
v = Value(x)

wei = q @ k.transpose(-2, -1)  # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

# Masking
tril = torch.tril(torch.ones(B, T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

out = wei @ v

In [ ]:
out.shape

torch.Size([4, 8, 16])